In [1]:
import pandas as pd
import numpy as np
import torch

from sklearn.metrics import f1_score,  precision_score, recall_score, hamming_loss

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import evaluate

c:\Users\su.osmond\AppData\Local\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the datasets
data_path = "../dataset"
df_test_texts = pd.read_csv(f'{data_path}/test_texts.csv', delimiter=";")
df_y_test = pd.read_csv(f'{data_path}/y_test.csv', delimiter=";")

print(df_test_texts.shape, df_y_test.shape)
df_test_texts.head()

(597, 1) (597, 74)


,text
0,Er is een teek op mijn rug en ik krijg hem er ...
1,huid
2,Er loopt bloed uit mijn oren
3,Ik heb al een week lang last van mijn oren
4,Al 3 dagen last van enorme tandpijn! Nu is de ...


In [3]:
complaints_to_index = {complaint: index for index, complaint in enumerate(df_y_test.columns)}
len(complaints_to_index)

74

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f"../models/trf/model_trf")
tokenizer = AutoTokenizer.from_pretrained(f"../models/trf/tokenizer_trf")

In [4]:
# df to numpy
test_texts = df_test_texts['text'].values
y_test = df_y_test.values

test_texts.shape, y_test.shape

((597,), (597, 74))

In [ ]:
input_text = test_texts[4]
print(input_text)
inputs = tokenizer(input_text, return_tensors="pt")

ik heb al een week lang last van mijn oren


In [ ]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
predictions = (torch.sigmoid(logits).numpy() > 0.5).astype(int)
predictions

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
predicted_labels = []

for idx, label in enumerate(df_y_test.columns):
    if predictions[0][idx]:
        predicted_labels.append(label)

predicted_labels

['Oorklachten']